Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML: https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Beautiful Soup

**Beautiful Soup** es una librería **Python** que permite extraer información de contenido en formato **HTML o XML**. Para usarla, es necesario especificar un **parser**, que es responsable de transformar un documento HTML o XML en un árbol complejo de objetos Python. Esto permite, por ejemplo, que podamos interactuar con los elementos de una página web como si estuviésemos utilizando las herramientas del desarrollador de un navegador.

A la hora de extraer información de una web, uno de los parsers más utilizado es el parser HTML de **lxml**. Precisamente, será el que utilicemos en este tutorial.

**Será necesario instalar las siguientes librerías** (si no las tienes ya):

        pip3 install beautifulsoup4 requests pandas

        pip3 install beautifulsoup4 

        pip3 install requests

        pip3 install pandas 

###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

Algunos ejemplos de Web Scraping utilizando Beautiful Soup:

https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/

http://omz-software.com/pythonista/docs/ios/beautifulsoup_guide.html

https://towardsdatascience.com/top-5-beautiful-soup-functions-7bfe5a693482

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [1]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html
import numpy as np
import lxml

## Caso 1: Scraping de un catálogo: Labirratorium

In [ ]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [ ]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página -> 958 cervezas 

Hacemos la consulta (request) y creamos la SOPA (como se denomina comunmente al XML/diccionario que se recibe -string en bruto sin estructura-) inicial:

In [2]:
pages = np.arange(1,81)
count = 1
lista_cervezas = []

for page in pages:

    # Accedemos a todo el contenido de la página
    URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page=' + str(page)
    r = requests.get (URL)
    soup = BeautifulSoup(r.text, 'lxml') # primer argumento es el texto de la consulta y el segundo 'lxml' es el formato para parsear el texto

    cervezas_grid = soup.find_all(class_ = 'product-image') # se acceden a todos los elementos de la sopa que estén en el tag 'product-image'

    count_beer = 1
    for cerveza in cervezas_grid:
        print('Cerveza {} de {}, pag {}/{}'.format(count_beer, len(cervezas_grid), page, len(pages)))
        id_cerv = 'lbt_' + str(count)

        # Hacemos un nuevo request para cada cerveza: 
        r = requests.get(cerveza.find('a')['href'])
        soup_cerve = BeautifulSoup(r.text, 'lxml')
        nombre_cerve = soup_cerve.find(class_ = 'h1 product-detail-name').text            # Nombre
        precio_cerve = soup_cerve.find(class_ = 'current-price').find('span')['content'] # 'span' es un tag dentro de la clase 'current-prices', hay que hacer un 'find'
        try:
            descshort_cerve = soup_cerve.find(class_ = 'description-short').find('p').text    # Descripción corta
        except:
            descshort_cerve = None

        try:
            # Descripción larga
            desclong_cerve = soup_cerve.find(class_ = 'product-description').find('p').text
        except:
            desclong_cerve = None

        try:
            # Brand
            marca = soup_cerve.find(class_ = 'img img-thumbnail manufacturer-logo')['alt']
        except:
            marca = None
        try:
            # Imagen
            imagen = soup_cerve.find(class_ = 'js-qv-product-cover img-fluid')['src']
        except:
            imagen = None 
        # Features
        # como sacar una tabla del estilo clave-valor con un diccionario
        tabla_dict = {}
        try:
            tabla = soup_cerve.find(class_ = 'data-sheet') # elemento con clave(dt) - valor(dd)
            tabla_dt_claves = tabla.find_all('dt')
            tabla_dd_valores = tabla.find_all('dd')
            for tabla, value in zip (tabla_dt_claves, tabla_dd_valores):    # se genera un diciconario con los tandem de valores obtenidos del elemento data-sheet
                tabla_dict[tabla.text] = value.text
        except:
            tabla_dict = {}
        lista_cervezas.append([
            ('id_cerv', id_cerv), 
            ('name', nombre_cerve),
            ('price', precio_cerve),
            ('desc_short', descshort_cerve),
            ('desclong_cerve', desclong_cerve),
            ('image', imagen), 
            ('brand', marca),
            ('features', tabla_dict)
        ])

        count += 1
        count_beer += 1

rveza 9 de 12, pag 19/80
Cerveza 10 de 12, pag 19/80
Cerveza 11 de 12, pag 19/80
Cerveza 12 de 12, pag 19/80
Cerveza 1 de 12, pag 20/80
Cerveza 2 de 12, pag 20/80
Cerveza 3 de 12, pag 20/80
Cerveza 4 de 12, pag 20/80
Cerveza 5 de 12, pag 20/80
Cerveza 6 de 12, pag 20/80
Cerveza 7 de 12, pag 20/80
Cerveza 8 de 12, pag 20/80
Cerveza 9 de 12, pag 20/80
Cerveza 10 de 12, pag 20/80
Cerveza 11 de 12, pag 20/80
Cerveza 12 de 12, pag 20/80
Cerveza 1 de 12, pag 21/80
Cerveza 2 de 12, pag 21/80
Cerveza 3 de 12, pag 21/80
Cerveza 4 de 12, pag 21/80
Cerveza 5 de 12, pag 21/80
Cerveza 6 de 12, pag 21/80
Cerveza 7 de 12, pag 21/80
Cerveza 8 de 12, pag 21/80
Cerveza 9 de 12, pag 21/80
Cerveza 10 de 12, pag 21/80
Cerveza 11 de 12, pag 21/80
Cerveza 12 de 12, pag 21/80
Cerveza 1 de 12, pag 22/80
Cerveza 2 de 12, pag 22/80
Cerveza 3 de 12, pag 22/80
Cerveza 4 de 12, pag 22/80
Cerveza 5 de 12, pag 22/80
Cerveza 6 de 12, pag 22/80
Cerveza 7 de 12, pag 22/80
Cerveza 8 de 12, pag 22/80
Cerveza 9 de 12, pag 

In [ ]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD


In [7]:
# crear un dataframe con la lista de listas obtenida
df = pd.DataFrame ([[x[0][1], 
                    x[1][1],
                    x[2][1],
                    x[3][1],
                    x[4][1],
                    x[5][1],
                    x[6][1],
                    x[7][1]] for x in lista_cervezas], columns= ['id', 'name', 'price', 'desc_short', 'desc_long', 'image', 'brand', 'features'])
df.to_csv('df_lista_cervezas_labirratorium.csv', sep=';')
df

,id,name,price,desc_short,desc_long,image,brand,features
0,lbt_1,"Boon Oude Kriek 37,5cl",7.15,Lambic / Kriek,Cerveza de fermentación espontánea (Lambic) de...,https://www.labirratorium.com/19351-large_defa...,Brouwerij F. Boon,"{'Estilo': 'KRIEK', 'Origen': 'Bélgica', '% Al..."
1,lbt_2,Störtebeker Schwarz-Bier,2.4,Cerveza negra de baja fermentación,Cerveza negra de baja fermentación de estilo S...,https://www.labirratorium.com/488-large_defaul...,Störtebeker,"{'Estilo': 'SCHWARZBIER', 'Origen': 'Alemania'..."
2,lbt_3,Orval,2.8,Belgian Pale Ale,Orval es una cerveza tipo ale y es la que prin...,https://www.labirratorium.com/385-large_defaul...,Orval,"{'Estilo': 'BELGIAN PALE ALE', 'Origen': 'Bélg..."
3,lbt_4,Augustiner Lagerbier Hell,2.5,Lager / Helles,Cerveza de baja fermentación de estilo Munich ...,https://www.labirratorium.com/367-large_defaul...,Augustiner,"{'Estilo': 'MUNICH HELLES', 'Origen': 'Alemani..."
4,lbt_5,Schneider Aventinus Weizen-Eisbock,2.7,"Cerveza de color medio oscuro, aromas alcohóli...","Cerveza de color medio oscuro, aromas alcohóli...",https://www.labirratorium.com/366-large_defaul...,Schneider,"{'Origen': 'Alemania', '% Alc.': '12', 'Volume..."
...,...,...,...,...,...,...,...,...
953,lbt_954,St. Peters Without GOLD Sin Alcohol 0.0,2.8,Sin Alcohol / Blonde Ale,"Cerveza sin alcohol (o,o% ABV) dorada y refres...",https://www.labirratorium.com/12551-large_defa...,St Peters,"{'Origen': 'Inglaterra', '% Alc.': '0.0% SIN A..."
954,lbt_955,Pohjala Prenzlauer 0,3.6,Sin Alcohol / Fruit Berliner Weisse,Cerveza afrutada y ligeramente ácida sin alcoh...,https://www.labirratorium.com/22380-large_defa...,Pohjala (Põhjala),"{'Estilo': 'SIN ALCOHOL', 'Origen': 'Estonia',..."
955,lbt_956,Lervig No Worries Lemon,2.9,Sin Alcohol / Fruit IPA,IPA sin alcohol (0.5% ABV) elaborada con limón...,https://www.labirratorium.com/21954-large_defa...,Lervig,"{'Estilo': 'SIN ALCOHOL', 'Origen': 'Noruega',..."
956,lbt_957,Mikkeller Baghaven Oud Sasughaven Blend 1,21.9,Lambic / Saison,Blend de Saison con Lambic de 2 y 3 años en co...,https://www.labirratorium.com/22353-large_defa...,Mikkeller Baghaven,"{'Estilo': 'GUEUZE', 'Origen': 'Dinamarca / Bé..."


Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [ ]:
# Agregamos a una lista


### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

### FBI: Top ten criminals

#### Queremos guardar las imágenes de cada fugitivo y que el nombre de cada archivo sea el nombre del fugitivo:

In [ ]:
fbi_url = 'https://www.fbi.gov/wanted/topten'

